In [5]:
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [123]:
house_df = pd.read_csv("../data/house_combined.csv")
suburb_list = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [124]:
df2 = house_df[['surburb']].drop_duplicates().reset_index().drop('index', axis = 1)

In [125]:
suburb_postcode_df = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [ ]:

list_1 = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        buildings = ox.geometries_from_place(place_name, tags = {'amenity': True, 'landuse':['retail','commercial']})
    except Exception:
        list_1[suburb] = None
        continue
    if len(buildings) == 0:
        list_1[suburb] = None
        continue
    for amenity, sub_df in buildings.groupby('amenity'):
        l[amenity] = len(sub_df)
    list_1[suburb] = l    

In [157]:
attribute_list = {**list_1, **list_2}
attribute_df = pd.DataFrame(attribute_list).T
attribute_df.to_csv('../data/raw/attribute_df.csv')
attribute_df = attribute_df[['bench','cafe','childcare','fast_food','kindergarten','parking','restaurant','place_of_worship','school','university']]

In [158]:
attribute_df

,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university
south melbourne,11.0,41.0,4.0,17.0,3.0,30.0,40.0,6.0,1.0,NaN
port melbourne,45.0,32.0,1.0,20.0,1.0,118.0,16.0,3.0,3.0,NaN
albert park,139.0,144.0,6.0,59.0,7.0,289.0,172.0,24.0,12.0,2.0
middle park,7.0,5.0,NaN,2.0,2.0,NaN,4.0,1.0,1.0,NaN
east melbourne,102.0,19.0,NaN,1.0,1.0,55.0,8.0,10.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
tallangatta,NaN,3.0,NaN,1.0,NaN,4.0,NaN,1.0,3.0,NaN
eskdale,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.0,1.0,NaN
dartmouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
beechworth,1.0,6.0,NaN,NaN,1.0,31.0,4.0,3.0,4.0,NaN


In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list_2 = {}
for suburb in suburb_list['surburb'][500:]:
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list_2[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list_2[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list_2[suburb] = l    

In [376]:
leisure_attribute_list = {**leisure_list, **leisure_list_2}
leisure_attribute_list = pd.DataFrame(leisure_attribute_list).T
leisure_attribute_list.to_csv('../data/raw/leisure_df.csv')
leisure_attribute_list = leisure_attribute_list[['garden','park','playground','pitch']]

In [399]:
attribute_conbined = pd.concat([attribute_df,leisure_attribute_list ], axis = 1)

In [400]:
attribute_conbined = attribute_conbined.reset_index().join(suburb_postcode_df).drop('index',axis = 1)
attribute_conbined.to_csv('../data/attribute_conbined.csv')

Download Australia CPI data


In [62]:
xls = pd.ExcelFile('../data/cpi.xlsx')
df1 = pd.read_excel(xls, 'Data')[13:].drop('Unnamed: 0', axis = 1).rename({'Unnamed: 1':'year','Unnamed: 2':'cpi'},axis = 1).reset_index().drop('index',axis = 1)
cpi_list = list(df1['cpi'])
l = []
m = 0
for i in range(2019,2026):
    l.append([i,cpi_list[m]])
    m = m+1
cpi_df = pd.DataFrame(l, columns=['year','cpi'])
cpi_df.to_csv('../data/cpi.csv')

In [12]:
import pandas as pd
import openrouteservice
import math
house_df_combined = pd.read_csv('../data/house_combined.csv')
import ast
lat_list = []
long_list = []
for i in range(len(house_df_combined)):
    if pd.isnull(house_df_combined.geoLocation[i]):
        lat_list.append(None)
        long_list.append(None)
    else:
        latitude = ast.literal_eval(house_df_combined.geoLocation[i])['latitude']
        longitude = ast.literal_eval(house_df_combined.geoLocation[i])['longitude']
        lat_list.append(latitude)
        long_list.append(longitude)
house_df_combined['latitude'] = lat_list
house_df_combined['longitude'] = long_list

In [13]:
import openrouteservice
import math
# melbourne CBD coordinate (144.961602,-37.813937)
distance_list_6 = []
for i in range(70000,72000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_6.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf624852e932154fff48e28bf173d51e105eca') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_6.append(distance)

46947.0
70274
52010.6
70275
54159.4
70276
51964.6
70277
49912.1
70278
51074.9
70279
56456.1
70280
56657.1
70281
44828.5
70282
56574.8
70283
56258.5
70284
47858.0
70285
56534.8
70286
58070.2
70287
58011.8
70288
51227.0
70289
56855.9
70290
56824.5
70291
54528.8
70292
57667.6
70293
55084.5
70294
51268.2
70295
50532.6
70296
34168.6
70297
49491.2
70298
48187.3
70299
48592.3
70300
54326.1
70301
49567.9
70302
51930.8
70303
52923.6
70304
43412.2
70305
50026.1
70306
48691.1
70307
56433.9
70308
51910.4
70309
28484.8
70310
52556.6
70311
42594.7
70312
51349.9
70313
41118.7
70314
50780.5
70315
51067.7
70316
57137.7
70317
56477.5
70318
53109.5
70319
51041.0
70320
56426.8
70321
50846.5
70322
49716.2
70323
52999.2
70324
54157.7
70325
42447.6
70326
51279.6
70327
51998.8
70328
52284.8
70329


ProxyError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/driving-car/json (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))

In [14]:
distance_list_7 = []
for i in range(70280,72000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_7.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf624852e932154fff48e28bf173d51e105eca') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_7.append(distance)

70280


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


56657.1
70281
44828.5
70282


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyon

56574.8
70283
56258.5
70284
47858.0
70285
56534.8
70286


...

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 8th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


54326.1
70301
49567.9
70302
51930.8
70303
52923.6
70304
43412.2
70305
50026.1
70306
48691.1
70307
56433.9
70308
51910.4
70309
28484.8
70310
52556.6
70311
42594.7
70312
51349.9
70313
41118.7
70314
50780.5
70315
51067.7
70316
57137.7
70317
56477.5
70318
53109.5
70319
51041.0
70320
56426.8
70321
50846.5
70322
49716.2
70323
52999.2
70324
54157.7
70325
42447.6
70326
51279.6
70327
51998.8
70328
52284.8
70329
61279.7
70330
55214.5
70331
56465.9
70332
44988.1
70333
55344.1
70334
47628.9
70335
39779.1
70336
43812.7
70337
45437.8
70338
53037.7
70339
51726.0
70340
53128.8
70341
24995.9
70342
55314.4
70343
49238.3
70344
56124.6
70345
49195.9
70346
55897.2
70347
43139.1
70348
49601.6
70349
54113.7
70350
57475.9
70351
51091.5
70352
52341.8
70353
45842.8
70354
50817.6
70355
50992.9
70356
48903.2
70357
35965.0
70358
48269.3
70359
34827.6
70360
42594.7
70361
51900.6
70362
45842.8
70363
55111.8
70364
56936.3
70365
48354.6
70366
40907.4
70367
62296.0
70368
34735.9
70369
41619.1
70370
42796.4
70371
56115.

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


53355.5
70376
50066.1
70377
47339.5
70378
49403.4
70379
43668.9
70380
46285.9
70381
50506.8
70382
49201.1
70383
45936.6
70384
52087.4
70385
47341.1
70386
55865.3
70387
50242.8
70388
42519.5
70389
41940.8
70390
53140.7
70391
51497.8
70392
34912.0
70393
52330.8
70394
45727.5
70395
55027.8
70396
50992.2
70397
50992.2
70398
32005.0
70399
50488.4
70400
49000.3
70401
19980.4
70402
54535.2
70403
56465.9
70404
53363.8
70405
59103.5
70406
59103.5
70407
50931.4
70408
50495.7
70409
65986.0
70410
55094.5
70411
53244.7
70412
53011.0
70413
53236.8
70414
53987.7
70415
48696.9
70416
50240.3
70417
50227.3
70418
45006.0
70419
44401.1
70420
47354.9
70421
53987.7
70422
52941.7
70423
50407.9
70424
51037.4
70425
42690.9
70426
48809.6
70427
49383.3
70428
43386.6
70429
36215.0
70430
53215.2
70431


ProxyError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/driving-car/json (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))

In [15]:
distance_list_8 = []
for i in range(70431,72000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_8.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf624852e932154fff48e28bf173d51e105eca') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_8.append(distance)

70431


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


49585.5
70432
59103.5
70433
48966.8
70434
52941.6
70435
46242.0
70436
50243.3
70437


In [86]:
pd.DataFrame(distance_list_9+distance_list_10+distance_list_11).to_csv('../data/raw/distance_list39')